In [2]:
import numpy as np
from ipynb.fs.full.base import graph_objects

In [20]:
parameter_pairs = []
for a in np.linspace(0.5,10):
    for b in np.linspace(0.5,10):
        parameter_pairs.append((a,b))

In [21]:
def beta_skew(a,b):
    return (2*(b-a)*np.sqrt(a+b+1))/((a+b+2)*np.sqrt(a*b))

In [26]:
#test to see what functions work:
parameter_cs = []
for pair in parameter_pairs:
    sample_avg_cs = []
    for sample_sizes in np.array([200,500,1000,5000,10000,50000]):
        cs = []
        for seed in np.array([2,34,567,2021,0,99,101,5000]):
            rng = np.random.default_rng(seed=seed)
            sample = rng.beta(pair[0],pair[1],sample_sizes) 
            test_sample = sample[sample >= np.percentile(sample, 66.7)]
            c = ((1-np.max(test_sample)) / np.std(test_sample,ddof=1))
            cs.append(c)
        sample_avg_cs.append(np.average(cs))
    parameter_cs.append(np.average(sample_avg_cs))

In [27]:
graph_objects(parameter_pairs, parameter_cs)

e^(-c): too extreme
c/(a^n+b^n): too extreme
adjusting skew (both adding skew, multiplying skew and adding exponent to the power of skew): no effect